In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

import wandb
wandb.login()

True

In [3]:

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define transforms for preprocessing and data augmentation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Using device: cuda


In [4]:
sweep_config = {
    "name": "CNN - HyperParameter Search",
    "metric": {"name": "val_accuracy", 
            "goal": "maximize"},
    "method": "random",
    "parameters": {
        "learning_rate": {
            "values": [0.001, 0.005, 0.01]
            },
        "num_conv_layer": {
            "values": [3, 5, 7]
            },
        "num_conv_filter": {
            "values": [16, 32]
            },
        "filter_org":{
            "values": [0.5, 1, 2]
        },
        "batch_norm": {
            "values": [True, False]
            },
        "dropout": {
            "values": [0.0, 0.2, 0.3, 0.4]
            },
        "num_dense_layers": {
            "values": [1, 2]
            },
        "dense_neurons": {
            "values": [256, 512]
            },
        "kernel_size":{
            "values":[3, 5]
        },
        "activation_func":{
            "values": ["relu", "gelu", "silu", "mish", "tanh"]
        }
    }
}

In [5]:
dataset_path = "inaturalist_12K/"
train_data = datasets.ImageFolder(root=f"{dataset_path}/train", transform=transform)
test_data = datasets.ImageFolder(root=f"{dataset_path}/val", transform=transform)

train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_subset, val_subset = random_split(train_data, [train_size, val_size])

# Define DataLoaders
batch_size = 64

# Loading data onto ram
class PreloadDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.data = [(img, label) for img, label in dataset]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_subset = PreloadDataset(train_subset)
val_subset = PreloadDataset(val_subset)
test_data = PreloadDataset(test_data)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"Training set size: {len(train_subset)}")
print(f"Validation set size: {len(val_subset)}")
print(f"Test set size: {len(test_data)}")

Training set size: 7999
Validation set size: 2000
Test set size: 2000


In [6]:

class CNN(nn.Module):
    def __init__(self,
                num_conv_layer=5,
                num_conv_filter=32,
                filter_org=2,
                batch_norm=False,
                dropout=0.25,
                kernel_size=3,
                activation_func='relu',
                num_dense_layers=1,
                dense_neurons=512):
        super().__init__()
        self.cnn_blocks = nn.Sequential()
        self.conv_output_dim = None
        
        input_channel = 3
        k = 0 # To keep track of the depth of the last conv layer
        output_channel = num_conv_filter
        
        for _ in range(num_conv_layer):
            layers = [nn.Conv2d(input_channel, output_channel, kernel_size, padding=(kernel_size-1)//2)]
            if batch_norm:
                layers.append(nn.BatchNorm2d(output_channel))
            if activation_func == 'relu':
                layers.append(nn.ReLU())
            elif activation_func == 'gelu':
                layers.append(nn.GELU())
            elif activation_func == 'silu':
                layers.append(nn.SiLU())
            elif activation_func == 'mish':
                layers.append(nn.Mish())
            elif activation_func == 'tanh':
                layers.append(nn.Tanh())
            
                
            layers.append(nn.MaxPool2d((2, 2)))
            self.cnn_blocks.append(nn.Sequential(*layers))
            
            input_channel = output_channel
            K = output_channel
            output_channel = int(output_channel * filter_org)

            if output_channel == 0:
                output_channel = 1
        
        conv_flatten = int(256/ 2**num_conv_layer) # The images are resized to 256 x 256
        self.conv_output_dim = conv_flatten*conv_flatten*K
        
        self.dense_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.conv_output_dim, dense_neurons),
            nn.Tanh(),
            # *[nn.Linear(dense_neurons, dense_neurons) for _ in range(num_dense_layers - 1)],
            *[nn.Sequential(nn.Linear(dense_neurons, dense_neurons), nn.Tanh()) for _ in range(num_dense_layers - 1)],
            nn.Dropout(dropout),
            nn.Linear(dense_neurons, 10)
        )

    
    def forward(self, x):
        x = self.cnn_blocks(x)
        x = x.view(x.size(0), -1)
        x = self.dense_layer(x)
        return x
        


In [7]:
def sweep_hyperparameters():
    default_config = {
        "learning_rate": 0.001,
        "num_conv_layer": 5,
        "num_conv_filter": 32,
        "filter_org": 1,
        "batch_norm": False,
        "dropout": 0.25,
        "kernel_size": 3,
        "activation_func": 'relu',
        "num_dense_layers": 1,
        "dense_neurons": 512
    }
    
    wandb.init(config = default_config)
    config = wandb.config # To get the Hyper Parameters from sweep_config
    
    learning_rate = config.learning_rate
    dense_neurons = config.dense_neurons
    num_dense_layers = config.num_dense_layers
    activation_func = config.activation_func
    kernel_size = config.kernel_size
    dropout = config.dropout
    batch_norm = config.batch_norm
    filter_org = config.filter_org
    num_conv_filter = config.num_conv_filter
    num_conv_layer = config.num_conv_layer
    
    run_name = f"Conv_Blocks: {num_conv_layer}, Num_Filters:{num_conv_filter}, Filter_org:{filter_org}, Kernel_size:{kernel_size}, AFunc: {activation_func}, Dense_Layers:{num_dense_layers}, Neurons_Dense_layer:{dense_neurons}"
    print("Sweep Name: ",run_name)
    
    model = CNN(num_conv_layer, num_conv_filter, filter_org, batch_norm
                ,dropout, kernel_size, activation_func, num_dense_layers, dense_neurons).to(device)
    model.to(device)
    
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    print(f"Model is on: {next(model.parameters()).device}")

    epochs = 10
    for epoch in range(epochs):
        print(f"\nEpoch [{epoch+1}/{epochs}]")

        # Training Phase
        model.train()
        train_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        avg_train_loss = train_loss / len(train_loader)

        # Validation Phase
        model.eval()
        val_loss, correct, total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        avg_val_loss = val_loss / len(val_loader)

        print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.2f}% | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")
        #print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")
        
        wandb.log({
            "epoch": epoch,
            "train_loss": avg_train_loss, 
            "train_accuracy": train_accuracy,
            "val_loss": avg_val_loss, 
            "val_accuracy": val_accuracy})
    wandb.run.name = run_name
    wandb.run.save()
    

sweep_ID = wandb.sweep(sweep_config, entity="manoj_da24s018-iit-madras", project="CNN-inaturalist_12K-HyperParameterSweep-Trail1")
wandb.agent(sweep_ID, sweep_hyperparameters, count=20)

Create sweep with ID: 8301demb
Sweep URL: https://wandb.ai/manoj_da24s018-iit-madras/CNN-inaturalist_12K-HyperParameterSweep-Trail1/sweeps/8301demb


wandb: Agent Starting Run: ljj2a8rq with config:
wandb: 	activation_func: silu
wandb: 	batch_norm: True
wandb: 	dense_neurons: 256
wandb: 	dropout: 0.2
wandb: 	filter_org: 2
wandb: 	kernel_size: 5
wandb: 	learning_rate: 0.005
wandb: 	num_conv_filter: 16
wandb: 	num_conv_layer: 5
wandb: 	num_dense_layers: 1


Sweep Name:  Conv_Blocks: 5, Num_Filters:16, Filter_org:2, Kernel_size:5, AFunc: silu, Dense_Layers:1, Neurons_Dense_layer:256
Model is on: cuda:0

Epoch [1/10]
Train Loss: 2.3848 | Train Acc: 14.19% | Val Loss: 2.2796 | Val Acc: 14.60%

Epoch [2/10]
Train Loss: 2.3456 | Train Acc: 14.18% | Val Loss: 2.3294 | Val Acc: 14.35%

Epoch [3/10]
Train Loss: 2.3247 | Train Acc: 15.06% | Val Loss: 2.2892 | Val Acc: 19.30%

Epoch [4/10]
Train Loss: 2.2800 | Train Acc: 16.81% | Val Loss: 2.2627 | Val Acc: 19.05%

Epoch [5/10]
Train Loss: 2.2934 | Train Acc: 16.76% | Val Loss: 2.2028 | Val Acc: 18.30%

Epoch [6/10]
Train Loss: 2.2484 | Train Acc: 17.80% | Val Loss: 2.2161 | Val Acc: 18.60%

Epoch [7/10]
Train Loss: 2.2498 | Train Acc: 17.83% | Val Loss: 2.1662 | Val Acc: 20.50%

Epoch [8/10]
Train Loss: 2.2547 | Train Acc: 18.61% | Val Loss: 2.1851 | Val Acc: 21.20%

Epoch [9/10]
Train Loss: 2.2285 | Train Acc: 19.79% | Val Loss: 2.1918 | Val Acc: 19.85%

Epoch [10/10]


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Train Loss: 2.2233 | Train Acc: 19.80% | Val Loss: 2.4018 | Val Acc: 17.60%


...

Sweep Name:  Conv_Blocks: 5, Num_Filters:32, Filter_org:1, Kernel_size:3, AFunc: mish, Dense_Layers:2, Neurons_Dense_layer:256
Model is on: cuda:0

Epoch [1/10]
Train Loss: 2.0788 | Train Acc: 26.17% | Val Loss: 2.0068 | Val Acc: 28.85%

Epoch [2/10]
Train Loss: 1.9218 | Train Acc: 32.33% | Val Loss: 1.9213 | Val Acc: 32.10%

Epoch [3/10]
Train Loss: 1.8455 | Train Acc: 35.20% | Val Loss: 2.0378 | Val Acc: 29.50%

Epoch [4/10]
Train Loss: 1.7817 | Train Acc: 37.42% | Val Loss: 1.8633 | Val Acc: 34.45%

Epoch [5/10]
Train Loss: 1.7206 | Train Acc: 39.87% | Val Loss: 1.8109 | Val Acc: 35.45%

Epoch [6/10]
Train Loss: 1.6456 | Train Acc: 42.89% | Val Loss: 1.8398 | Val Acc: 35.50%

Epoch [7/10]
Train Loss: 1.5661 | Train Acc: 45.69% | Val Loss: 1.8619 | Val Acc: 35.65%

Epoch [8/10]
Train Loss: 1.5106 | Train Acc: 47.53% | Val Loss: 1.8527 | Val Acc: 37.25%

Epoch [9/10]
Train Loss: 1.4254 | Train Acc: 50.99% | Val Loss: 1.8357 | Val Acc: 37.25%

Epoch [10/10]
Train Loss: 1.3360 | Train A

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▃▄▄▅▆▆▇█
train_loss,█▇▆▅▅▄▃▃▂▁
val_accuracy,▁▄▂▆▇▇▇██▅
val_loss,▇▄█▃▁▂▃▂▂▆
epoch,9
train_accuracy,53.68171
train_loss,1.33604
val_accuracy,34.1
val_loss,1.95998


wandb: Agent Starting Run: jbmhv1l6 with config:
wandb: 	activation_func: tanh
wandb: 	batch_norm: True
wandb: 	dense_neurons: 512
wandb: 	dropout: 0.4
wandb: 	filter_org: 0.5
wandb: 	kernel_size: 3
wandb: 	learning_rate: 0.01
wandb: 	num_conv_filter: 32
wandb: 	num_conv_layer: 7
wandb: 	num_dense_layers: 1


Sweep Name:  Conv_Blocks: 7, Num_Filters:32, Filter_org:0.5, Kernel_size:3, AFunc: tanh, Dense_Layers:1, Neurons_Dense_layer:512
Model is on: cuda:0

Epoch [1/10]
Train Loss: 2.3552 | Train Acc: 13.06% | Val Loss: 2.3114 | Val Acc: 12.10%

Epoch [2/10]
Train Loss: 2.2693 | Train Acc: 14.75% | Val Loss: 2.3435 | Val Acc: 11.50%

Epoch [3/10]
Train Loss: 2.2474 | Train Acc: 15.60% | Val Loss: 2.2354 | Val Acc: 15.45%

Epoch [4/10]
Train Loss: 2.2263 | Train Acc: 16.14% | Val Loss: 2.2257 | Val Acc: 16.40%

Epoch [5/10]
Train Loss: 2.2202 | Train Acc: 17.11% | Val Loss: 2.2500 | Val Acc: 14.05%

Epoch [6/10]
Train Loss: 2.1985 | Train Acc: 17.54% | Val Loss: 2.1980 | Val Acc: 18.75%

Epoch [7/10]
Train Loss: 2.1720 | Train Acc: 19.48% | Val Loss: 2.1765 | Val Acc: 17.95%

Epoch [8/10]
Train Loss: 2.1564 | Train Acc: 19.93% | Val Loss: 2.1544 | Val Acc: 19.65%

Epoch [9/10]
Train Loss: 2.1433 | Train Acc: 20.48% | Val Loss: 2.1563 | Val Acc: 19.20%

Epoch [10/10]
Train Loss: 2.1423 | Train

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▃▄▅▅▇▇██
train_loss,█▅▄▄▄▃▂▁▁▁
val_accuracy,▂▁▄▅▃▇▇██▇
val_loss,▇█▄▄▅▃▂▁▁▂
epoch,9
train_accuracy,20.50256
train_loss,2.14231
val_accuracy,18.9
val_loss,2.17486
